<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze-.iframe.experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
from IPython.display import IFrame

IFrame(src='https://voodoohop.github.io/colabasaservice/', width=700, height=600)

In [42]:
!npm install -g http-server
!nohup http-server -p 8000 &

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

/tools/node/bin/http-server -> /tools/node/lib/node_modules/http-server/bin/http-server
/tools/node/bin/hs -> /tools/node/lib/node_modules/http-server/bin/http-server
+ http-server@0.12.3
updated 1 package in 0.972s
nohup: appending output to 'nohup.out'
https://o6f9f459ozp-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [ ]:
!pip install deep-daze stomp.py imageio-ffmpeg --upgrade

In [ ]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import stomp
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 

flatten = itertools.chain.from_iterable

TEXT="A technical illustration of an upgraded cat made of cyberpunk parts"

DEFAULT_PARAMS='--lower_bound_cutout 0.4 --image-width=512 --iterations=5000 --epochs=2 --save-every=2 --save-progress=True --open_folder=False --overwrite=True --save_video=True'

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)



def run(text, params=DEFAULT_PARAMS, watch_media_paths=["./*.jpg","./*.mp4"]):
  
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  cmd = f'imagine {params} "{text}"'
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)

    headers = { 
        "params": params,
        "text": TEXT
    }
    conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files

    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          filename = os.path.basename(added)
          mimetype, _ = guess_type(filename)
          print("Adding:", added,"type",mimetype)
          
          with open(added,"rb") as media_file:
            added_media = media_file.read()

            base64_encoded_media = f"data:{mimetype};base64,{b64encode(added_media).decode('ascii')}"
          
          headers = {
              **headers,         
              "type": mimetype,
              "filename": filename
              }
          conn.send(body=base64_encoded_media, headers=headers, destination='/topic/colabMediaOut')

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))
run(TEXT)
conn.disconnect()



epochs:   0%|          | 0/2 [00:00<?, ?it/s]
                                             
Adding: ./A_technical_illustration_of_an_upgraded_cat_made_of_cyberpunk_parts.jpg type image/jpeg
loss: -20.27:   0%|          | 1/5000 [00:00<35:41,  2.33it/s]
                                             
Adding: ./A_technical_illustration_of_an_upgraded_cat_made_of_cyberpunk_parts.000001.jpg type image/jpeg
loss: -20.37:   0%|          | 3/5000 [00:01<34:07,  2.44it/s]
                                             
Adding: ./A_technical_illustration_of_an_upgraded_cat_made_of_cyberpunk_parts.000002.jpg type image/jpeg
loss: -21.62:   0%|          | 5/5000 [00:02<33:58,  2.45it/s]
                                             
Adding: ./A_technical_illustration_of_an_upgraded_cat_made_of_cyberpunk_parts.000003.jpg type image/jpeg
loss: -22.87:   0%|          | 7/5000 [00:03<33:48,  2.46it/s]
                                             
Adding: ./A_technical_illustration_of_an_upgraded_cat_made_

# New section

In [ ]:
%%javascript
// Create the channel object
const channel = new BroadcastChannel('my-channel');

// Set up a listener
channel.onmessage = message => console.log('Received:', message.data);

// Post a message to all subscribers
channel.postMessage('Hello, world');

In [ ]:
conn.disconnect()

In [43]:
!pip install simple-websocket-server

  Created wheel for simple-websocket-server: filename=simple_websocket_server-0.4.1-cp37-none-any.whl size=8766 sha256=2b5f383d1395956e33c276d8bb556c5d5cfa44c681d05620f534ee61d4a311fc
  Stored in directory: /root/.cache/pip/wheels/fb/a6/fd/6a5350f4381f87617e874be45ff7620e6cb03219684cf0a88e
Successfully built simple-websocket-server


In [45]:
from simple_websocket_server import WebSocketServer, WebSocket


class SimpleEcho(WebSocket):
    def handle(self):
        # echo message back to client
        self.send_message(self.data)

    def connected(self):
        print(self.address, 'connected')

    def handle_close(self):
        print(self.address, 'closed')


server = WebSocketServer('', 8000, SimpleEcho)
server.serve_forever()

KeyboardInterrupt: ignored